# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing graphlab functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in graphlab functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Fire up graphlab create

In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.cross_validation import train_test_split
from sklearn import linear_model

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [3]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 
                       'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 
                       'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':str, 'condition':int,
                       'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 
                       'view':int}
sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)

# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let GraphLab Create pick a random seed for you).  

In [4]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
id               21613 non-null object
date             21613 non-null object
price            21613 non-null float64
bedrooms         21613 non-null float64
bathrooms        21613 non-null float64
sqft_living      21613 non-null float64
sqft_lot         21613 non-null int64
floors           21613 non-null object
waterfront       21613 non-null int64
view             21613 non-null int64
condition        21613 non-null int64
grade            21613 non-null int64
sqft_above       21613 non-null int64
sqft_basement    21613 non-null int64
yr_built         21613 non-null int64
yr_renovated     21613 non-null int64
zipcode          21613 non-null object
lat              21613 non-null float64
long             21613 non-null float64
sqft_living15    21613 non-null float64
sqft_lot15       21613 non-null float64
dtypes: float64(8), int64(9), object(4)
memory usage: 3.5+ MB


In [5]:
#train_data, test_data = train_test_split(sales, train_size = 0.8, random_state = 0)

train_data = pd.read_csv('kc_house_train_data.csv', dtype = dtype_dict)
test_data = pd.read_csv('kc_house_test_data.csv', dtype = dtype_dict)

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [6]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
#example_model = graphlab.linear_regression.create(train_data, target = 'price', features = example_features, 
#                                                  validation_set = None)
example_model = linear_model.LinearRegression()

In [7]:
example_model = example_model.fit(train_data[example_features], train_data['price'])

Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [8]:
example_weight_summary = (example_model.intercept_, example_model.coef_)
print example_weight_summary

(87912.865814965044, array([   315.40669062, -65081.88711588,   6942.16598637]))


# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing graphlab create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [9]:
example_predictions = example_model.predict(train_data[example_features])
print example_predictions[0] # should be 271789.505878

271789.26538


# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [10]:
def get_residual_sum_of_squares(model, dataX, outcome):
    # First get the predictions
    predictions = model.predict(dataX)

    # Then compute the residuals/errors
    resids = outcome - predictions

    # Then square and add them up
    RSS = (resids * resids).sum()

    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [11]:
rss_example_train = get_residual_sum_of_squares(example_model, test_data[example_features], test_data['price'])
print rss_example_train # should be 2.7376153833e+14

2.73761940583e+14


# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [12]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [13]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

In [14]:
# create the remaining 3 features in both TEST and TRAIN data
train_data['bed_bath_rooms'] = train_data['bedrooms'] * train_data['bathrooms']
test_data['bed_bath_rooms'] = test_data['bedrooms'] * test_data['bathrooms']

train_data['log_sqft_living'] = train_data['sqft_living'].apply(log)
test_data['log_sqft_living'] = test_data['sqft_living'].apply(log)

train_data['lat_plus_long'] = train_data['lat'] + train_data['long']
test_data['lat_plus_long'] = test_data['lat'] + test_data['long']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

In [15]:
new_features = ['bedrooms_squared', 'bed_bath_rooms','log_sqft_living', 'lat_plus_long']
[round(test_data[f].mean(), 2) for f in new_features]

[12.45, 7.5, 7.55, -74.65]

In [16]:
log(2)

0.6931471805599453

# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [17]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using graphlab.linear_regression.create() and look at the value of the weights/coefficients:

In [18]:
# Learn the three models:
model_1 = linear_model.LinearRegression().fit(train_data[model_1_features], train_data['price'])
model_2 = linear_model.LinearRegression().fit(train_data[model_2_features], train_data['price'])
model_3 = linear_model.LinearRegression().fit(train_data[model_3_features], train_data['price'])

In [19]:
# Examine/extract each model's coefficients:
print "Intercept:",  model_1.intercept_,  "Coef: ", dict(zip(model_1_features, model_1.coef_)), "\n"
print "Intercept:",  model_2.intercept_,  "Coef: ", dict(zip(model_2_features, model_2.coef_)), "\n"
print "Intercept:",  model_3.intercept_,  "Coef: ", dict(zip(model_3_features, model_3.coef_)), "\n"

Intercept: -69075726.7926 Coef:  {'lat': 658619.26393051539, 'sqft_living': 312.25864627320271, 'bedrooms': -59586.533153612218, 'bathrooms': 15706.742082734667, 'long': -309374.35126823327} 

Intercept: -66867968.8711 Coef:  {'bathrooms': -71461.308292758695, 'bedrooms': -113446.36807020336, 'long': -294298.96913811879, 'sqft_living': 306.61005345899548, 'lat': 654844.62950330123, 'bed_bath_rooms': 25579.652000752052} 

Intercept: -62036084.9861 Coef:  {'bathrooms': 67060.781318911162, 'bedrooms': 34514.229577989994, 'long': -406750.71086103987, 'sqft_living': 529.42281964652261, 'lat_plus_long': 127334.90000645581, 'bedrooms_squared': -6788.5866703405109, 'lat': 534085.61086749472, 'log_sqft_living': -561831.48407552089, 'bed_bath_rooms': -8570.5043946301012} 



**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

*positive*

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

*negative*

Think about what this means.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [20]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
Train_RSS_model_1 = get_residual_sum_of_squares(model_1, train_data[model_1_features], train_data['price'])
Train_RSS_model_2 = get_residual_sum_of_squares(model_2, train_data[model_2_features], train_data['price'])
Train_RSS_model_3 = get_residual_sum_of_squares(model_3, train_data[model_3_features], train_data['price'])

In [21]:
print Train_RSS_model_1, Train_RSS_model_2, Train_RSS_model_3

9.6787996305e+14 9.58419635074e+14 9.0343645505e+14


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

*Ans: Model 3*

Now compute the RSS on on TEST data for each of the three models.

In [22]:
# Compute the RSS on TESTING data for each of the three models and record the values:
Test_RSS_model_1 = get_residual_sum_of_squares(model_1, test_data[model_1_features], test_data['price'])
Test_RSS_model_2 = get_residual_sum_of_squares(model_2, test_data[model_2_features], test_data['price'])
Test_RSS_model_3 = get_residual_sum_of_squares(model_3, test_data[model_3_features], test_data['price'])

In [23]:
print Test_RSS_model_1, Test_RSS_model_2, Test_RSS_model_3

2.25500469795e+14 2.23377462976e+14 2.59236319207e+14


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected?Think about the features that were added to each model from the previous.

*Ans: Model 2*